# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  + 1️⃣-***nuclei***
  + 2️⃣-***cellmask***
  + 3️⃣-***cytoplasm*** (+ ***nucleus***)
  + 4️⃣-***lysosome***
  + 5️⃣-***mitochondria***
  + 6️⃣-***golgi***
  + 7️⃣-***peroxisome***
  + 8️⃣-***ER***
  + 9️⃣-***lipid droplet***





## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A




## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops, label
from skimage.morphology import binary_erosion


import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.core.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    import_inferred_organelle,
                                                                    export_tiff,
                                                                    list_image_files)

from infer_subc_2d.core.img import *
from infer_subc_2d.utils.stats import *

from infer_subc_2d.organelles import * 

import time
%load_ext autoreload
%autoreload 2



In [2]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROX_CH ,
                                                                    ER_CH ,
                                                                    LD_CH ,
                                                                    RESIDUAL_CH )              


## SETUP

In [3]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

source_file = meta_dict['file_name']

/opt/anaconda3/envs/cento/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:
scale[0]/scale[1], scale

(7.267318290023735,
 (0.5804527163320905, 0.07987165184837318, 0.07987165184837318))

Now get the single "optimal" slice of all our organelle channels....

## get the inferred cellmask, nuclei and cytoplasm objects

(takes < 1 sec)

Builde the segmentations in order




In [6]:

###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
cellmask_obj = get_cellmask(img_data, nuclei_obj, meta_dict, out_data_path)
cyto_mask = get_cytoplasm(nuclei_obj , cellmask_obj , meta_dict, out_data_path)



`nuclei` object not found: /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-nuclei.tiff
starting segmentation...
 in_img = (8, 15, 768, 768)
nuclei size = (15, 768, 768)
>>>>>>>>>>>> tifffile.imwrite in (0.01) sec
saved file: None
inferred nuclei. wrote None
inferred nuclei in (7.24) sec
`cellmask` object not found: /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cellmask.tiff
starting segmentation...
shape in_img (8, 15, 768, 768)
shape nuclei_obj (15, 768, 768)
weighted_aggregate: shape in- (8, 15, 768, 768) , shape_out- (15, 768, 768)
shape struct_img (15, 768, 768)
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.02) sec
saved file: None
inferred (and exported) cellmask in (58.35) sec
`cytoplasm` object not found: /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cytoplasm.tiff
starting segmentation...
changed dtype from bool to uint8
>>>>>>>>>>>> tifffile.imwrite in (0.

-------------------------
## regionprops

`skimage.measure.regionprops` provides the basic tools nescessary to quantify our segmentations.

First lets see what works in 3D.  

> Note: the names of the regionprops correspond to the 2D analysis even for those which are well defined in 3D.  i.e. "area" is actually "volume" in 3D, etc.


-----------------
## basic stats

### per-organelle


- regionprops 


### summary stats

- group + aggregate:  surface_area, volume
  - median
  - mean
  - std 
  - count

- normalizers
  - SOMA?
  - CYTOSOL?

### nuclei caveats
The other organelles are sensibly normalized by cytoplasm.  does normalizing the nuclei by cytoplasm make sense?  or use cellmask?

Lets see which possible measures are sensible for 3D or volumetric with regionprops

In [7]:
labels = label(nuclei_obj )
rp = regionprops(labels, intensity_image=img_data[NUC_CH])

supported = [] 
unsupported = []

for prop in rp[0]:
    try:
        rp[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

Supported properties:
  area
  bbox
  bbox_area
  centroid
  convex_area
  convex_image
  coords
  equivalent_diameter
  euler_number
  extent
  feret_diameter_max
  filled_area
  filled_image
  image
  inertia_tensor
  inertia_tensor_eigvals
  intensity_image
  label
  local_centroid
  major_axis_length
  max_intensity
  mean_intensity
  min_intensity
  minor_axis_length
  moments
  moments_central
  moments_normalized
  slice
  solidity
  weighted_centroid
  weighted_local_centroid
  weighted_moments
  weighted_moments_central
  weighted_moments_normalized

Unsupported properties:
  eccentricity
  moments_hu
  orientation
  perimeter
  perimeter_crofton
  weighted_moments_hu


In [8]:
# from scipy.ndimage import find_objects
    
# labels = label(nuclei_obj ).astype("int")
# objects = find_objects(labels)

# # objects are the slices into the original array for each organelle

In [9]:
# get overall summary stats for cellmask
cm_intensity =  raw_cellmask_fromaggr(img_data, scale_min_max=False)


weighted_aggregate: shape in- (8, 15, 768, 768) , shape_out- (15, 768, 768)


In [10]:

def dump_stats(name: str, segmentation:np.ndarray, intensity_img:np.ndarray, mask:np.ndarray, out_data_path: Path, source_file: str) -> pd.DataFrame:
    """
    get summary stats
    calls `get_summary_stats_3D`
    """
    
    stats_table,_ = get_summary_stats_3D(segmentation, intensity_img, mask) 
    csv_path = out_data_path / f"{source_file.stem}-{name}-stats.csv"
    stats_table.to_csv(csv_path )
    print(f"dumped {name} table to {csv_path}")

    return stats_table

    


In [11]:
cellmask_table = dump_stats("cellmask", cellmask_obj, cm_intensity, cellmask_obj, out_data_path, source_file)
nucleus_table = dump_stats("nucleus", nuclei_obj, img_data[NUC_CH], cellmask_obj, out_data_path, source_file)
cyto_table = dump_stats("cytosol", cyto_mask, cm_intensity+img_data[NUC_CH], cellmask_obj, out_data_path, source_file)


dumped cellmask table to /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cellmask-stats.csv
dumped nucleus table to /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-nucleus-stats.csv
dumped cytosol table to /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-cytosol-stats.csv


In [12]:
cellmask_obj.dtype

dtype('uint8')

Now we want to get a list of our organelle names, segmentations, intensities (florescence)

In [7]:
# names of organelles we have
organelle_names = ["lyso", "mitochondria","golgi","peroxisome","er","lipid"]

get_methods  = [get_lyso,
            get_mito,
            get_golgi,
            get_perox,
            get_ER,
            get_LD]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

intensities = [img_data[ch] for ch in organelle_channels]


>>>>>>>>>>>> tifffile.imread  (dtype=uint16in (0.02) sec
loaded  inferred 3D `lyso`  from /Users/ahenrie/Projects/Imaging/data/out 
loaded lyso in (0.02) sec
`mitochondria` object not found: /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-mitochondria.tiff
starting segmentation...
>>>>>>>>>>>> tifffile.imwrite in (0.02) sec
saved file: None
inferred mitochondria. wrote None
inferred (and exported) mitochondria in (26.77) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16in (0.02) sec
loaded  inferred 3D `golgi`  from /Users/ahenrie/Projects/Imaging/data/out 
starting segmentation...
`peroxisome` object not found: /Users/ahenrie/Projects/Imaging/data/out/ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed-peroxisome.tiff
starting segmentation...
>>>>>>>>>>>> tifffile.imwrite in (0.02) sec
saved file: None
inferred peroxisome. wrote None
inferred (and exported) peroxisome in (2.57) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint16in (0.02) sec
loaded  inferred 3D `er`  from


-----------------
## CONTACTS (cross-stats)

### organelle cross stats


- regionprops 



- intersect for A vs all other organelles Bi
  - regionprops on A ∩ Bi

   
- contacts?
  - dilate then intersect?
  - loop through each sub-object for each 



In [14]:



def shell_cross_stats(organelle_names: List[str], organelles:List[np.ndarray], mask:np.ndarray, out_data_path: Path, source_file: str) -> int:
    """
    get all cross stats between organelles `a` and `b`, and "shell of `a`" and `b`.   "shell" is the boundary of `a` 
    calls `get_aXb_stats_3D`
    """
    count = 0
    for j, target in enumerate(organelle_names):    
        print(f"getting stats for A = {target}")
        a = organelles[j ]
        # loop over Bs
        for i,nmi in enumerate(organelle_names):    
            if  i  !=  j:
                # get overall stats of intersection
                print(f"  X {nmi}")
                b = organelles[i]
                stats_tab = get_aXb_stats_3D(a,b, mask)    
                csv_path = out_data_path / f"{source_file.stem}-{target}X{nmi}-stats.csv"
                stats_tab.to_csv(csv_path )

                e_stats_tab = get_aXb_stats_3D(a,b, cytoplasm_mask, use_shell_a=True)
                csv_path = out_data_path / f"{source_file.stem}-{target}_shellX{nmi}-stats.csv"
                e_stats_tab.to_csv(csv_path )
                
                count += 1
    return count

    


In [15]:

def organelle_stats(organelle_names: List[str], organelles:List[np.ndarray], intinsities:List[np.ndarray], mask:np.ndarray, out_data_path: Path, source_file: str) -> int:
    """
    get summary and all cross stats between organelles A and B

    calls `get_summary_stats_3D`
    """
    count = 0
    org_stats_tabs = []
    for j,target in enumerate(organelle_names):    
        print(f"getting stats for A = {target}")
        A = organelles[ j ]
        # A_stats_tab, rp = get_simple_stats_3D(A,mask)
        A_stats_tab, rp = get_summary_stats_3D(A , intinsities[ j ] ,mask )

        # loop over Bs
        for i,nmi in enumerate(organelle_names):    
            if  i != j:
                # get overall stats of intersection
                print(f"  b = {nmi}")
                B = organelles[i]

                count += 1 
                # add the list of touches 
                labB = label(B)

                ov = []
                B_labs = []
                labs = []
                for idx,lab in enumerate(A_stats_tab['label']): # loop over A_objects
                    xyz = tuple(rp[idx].coords.T)
                    cmp_org = labB[xyz]

                    #total number of overlapping pixels
                    overlap = sum(cmp_org>0)
                    # overlap?
                    b_labs = cmp_org[cmp_org>0]
                    b_js = np.unique(b_labs).tolist()

                    # if overlap > 0:
                    labs.append(lab)
                    ov.append(overlap)
                    B_labs.append(b_js)

                # add organelle B columns to A_stats_tab
                A_stats_tab[f"{nmi}_overlap"] = ov
                A_stats_tab[f"{nmi}_labels"] = B_labs  # might want to make this easier for parsing later

        # org_stats_tabs.append(A_stats_tab)
        csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"

        A_stats_tab.to_csv(csv_path)


    print(f"dumped {count} csvs")
    return count

    
# refactor to just to a target vs. list of probes
# for nuclei mask == cellmask
# for all oother mask == cytoplasm



In [16]:

def organelle_stats(organelle_names: List[str], organelles:List[np.ndarray], intinsities:List[np.ndarray], mask:np.ndarray, out_data_path: Path, source_file: str) -> int:
    """
    get summary and all cross stats between organelles A and B

    calls `get_summary_stats_3D`
    """
    count = 0
    org_stats_tabs = []
    for j,target in enumerate(organelle_names):    
        print(f"getting stats for `a` = {target}")
        a = organelles[ j ]
        # A_stats_tab, rp = get_simple_stats_3D(A,mask)
        a_stats_tab, rp = get_summary_stats_3D(a , intinsities[ j ] ,mask )

        # loop over Bs
        for i,nmi in enumerate(organelle_names):    
            if  i != j:
                # get overall stats of intersection
                print(f"  `b` = {nmi}")
                b = organelles[i]

                count += 1
                # in case we sent a boolean mask (e.g. cyto, nucleus, cellmask)
                if b.dtype == "bool" or b.dtype == np.uint8 :
                    lab_b = label(b > 0).astype(np.uint16)
                else:
                    lab_b = b
                ov = []

                b_labels = []
                for idx,lab in enumerate(a_stats_tab['label']): # loop over A_objects
                    xyz = tuple(rp[idx].coords.T)
                    cmp_org = lab_b[xyz]

                    #total number of overlapping pixels
                    overlap = sum(cmp_org>0)
                    # overlap?
                    b_ls = cmp_org[cmp_org>0]
                    b_js = np.unique(b_ls).tolist()

                    # if overlap > 0:
                    ov.append(overlap)
                    b_labels.append(b_js)

                # add organelle B columns to A_stats_tab
                a_stats_tab[f"{nmi}_overlap"] = ov
                a_stats_tab[f"{nmi}_labels"] = b_labels  # might want to make this easier for parsing later

        # org_stats_tabs.append(A_stats_tab)
        csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"

        a_stats_tab.to_csv(csv_path)


    print(f"dumped {count} csvs")
    return count

    
# refactor to just to a target vs. list of probes
# for nuclei mask == cellmask
# for all oother mask == cytoplasm



In [65]:
n_files = organelle_stats(organelle_names, organelles,intensities, cyto_mask, out_data_path, source_file)

getting stats for `a` = lyso
  `b` = mitochondria
  `b` = golgi
  `b` = peroxisome
  `b` = er
  `b` = lipid
getting stats for `a` = mitochondria
  `b` = lyso
  `b` = golgi
  `b` = peroxisome
  `b` = er
  `b` = lipid
getting stats for `a` = golgi
  `b` = lyso
  `b` = mitochondria
  `b` = peroxisome
  `b` = er
  `b` = lipid
getting stats for `a` = peroxisome
  `b` = lyso
  `b` = mitochondria
  `b` = golgi
  `b` = er
  `b` = lipid
getting stats for `a` = er
  `b` = lyso
  `b` = mitochondria
  `b` = golgi
  `b` = peroxisome
  `b` = lipid
getting stats for `a` = lipid
  `b` = lyso
  `b` = mitochondria
  `b` = golgi
  `b` = peroxisome
  `b` = er
dumped 30 csvs


In [66]:
n_files =shell_cross_stats(organelle_names, organelles, cyto_mask, out_data_path, source_file) 


getting stats for A = lyso
  X mitochondria
  X golgi
  X peroxisome
  X er
  X lipid
getting stats for A = mitochondria
  X lyso
  X golgi
  X peroxisome
  X er
  X lipid
getting stats for A = golgi
  X lyso
  X mitochondria
  X peroxisome
  X er
  X lipid
getting stats for A = peroxisome
  X lyso
  X mitochondria
  X golgi
  X er
  X lipid
getting stats for A = er
  X lyso
  X mitochondria
  X golgi
  X peroxisome
  X lipid
getting stats for A = lipid
  X lyso
  X mitochondria
  X golgi
  X peroxisome
  X er



-----------------
##  SUMMARY STATS  
> WARNING: (🚨🚨🚨🚨 WIP)
### normalizations.

- overlaps, normalized by CYTOSOL, A, and B
- per cell averages, medians, std, and totals

These is all pandas munging and very straightforward tabular manipulation.


In [17]:
target = organelle_names[1]

csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"

mito_table = pd.read_csv(csv_path)
mito_table.head()

,Unnamed: 0,label,max_intensity,mean_intensity,min_intensity,volume,equivalent_diameter,centroid-0,centroid-1,centroid-2,...,lyso_overlap,lyso_labels,golgi_overlap,golgi_labels,peroxisome_overlap,peroxisome_labels,er_overlap,er_labels,lipid_overlap,lipid_labels
0,0,1,18751,7867.069930,0,143,6.488024,1.000000,342.356643,523.419580,...,0,[],0,[],0,[],0,[],0,[]
1,1,3,61674,18140.592896,0,6475,23.124928,4.872587,405.907645,562.084324,...,302,"[33, 34, 57, 174, 209]",0,[],0,[],1565,"[5, 27, 78]",0,[]
2,2,5,26609,11076.260417,0,96,5.680992,1.000000,397.927083,481.802083,...,0,[],0,[],0,[],5,[19],0,[]
3,3,6,52794,15597.380997,0,50444,45.842712,6.627448,176.820613,482.624713,...,3906,"[7, 13, 41, 44, 76, 83, 106, 108, 109, 111, 11...",1042,"[1, 3, 14, 16]",120,"[4, 7, 14, 15, 18, 19, 20, 21, 24, 25]",13740,"[5, 115, 275, 307, 313, 344]",0,[]
4,4,7,7772,3231.764706,0,17,3.190192,1.000000,176.588235,523.176471,...,0,[],0,[],0,[],8,[5],0,[]


In [68]:
mito_table.volume.mean()

883.2333333333333


-----------------
## DISTRIBUTION  
> WARNING: (🚨🚨🚨🚨 WIP)
### XY- summary
Segment image in 3D;
sum projection of binary image; 
create 5 concentric rings going from the edge of the nuclie to the edge of the cellmask (ideally these will be morphed to cellmask/nuclei shape as done in CellProfiler); 
measure intensity per ring (include nuclei as the center area to measure from)/ring area; 
the normalized measurement will act as a frequency distribution of that organelle starting from the nuclei bin going out to the cell membrane - 
Measurements needed: mean, median, and standard deviation of the frequency will be calculated

- pre-processing
  1. Make 2D sum projection of binary segmentation
  2. Create 5 concentric rings going out from the edge of the nuclei to the edge of the cellmask - these rings should be morphed to the shape of the nuclei and cellmask. 
  3. Use nucleus + concentric rings to mask the 2D sum project into radial distribution regions: nuclei = bin 1, ... largest/outter most ring = bin 6. See similar concept in CellProfiler: https://cellprofiler-manual.s3.amazonaws.com/CellProfiler-4.2.5/modules/measurement.html?highlight=distribution#module-cellprofiler.modules.measureobjectintensitydistribution"	
   
- per-object measurements
  - For each bin measure:
    1. pixel ""intensity""
    2. bin area"

- per-object calculations
  - per-object For each bin: 
  - sum of pixel intensity per bin / bin area"	

- per cell summary
  1. Create a frequency table with bin number of the x axis and normalized pixel intensity on the y-axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"




### Z- summary
Segment image in 3D;
measure area fraction of each organelle per Z slice;
these measurements will act as a frequency distribution of that organelle starting from the bottom of the cellmask (not including neurites) to the top of the cellmask;
measurements: mean, median, and standard deviation of the frequency distribution	

- pre-processing
  1. subtract nuclei from the cellmask --> cellmask cytoplasm
  2. mask organelle channels with cellmask cytoplasm mask

- per-object measurements
  - For each Z slice in the masked binary image measure:
    1. organelle area
    2. cellmask cytoplasm area

- per-object calculations
  - For each Z slice in the masked binary image: organelle area / cellmask cytoplasm area

- per cell summary
  1. create a frequency table with the z slice number on the x axis and the area fraction on the y axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"

In [11]:

# get the flattened
_cellmask = cellmask_obj.sum(axis=0) 
_nuclei = apply_mask(nuclei_obj,cellmask_obj).sum(axis=0)
d_to_edge = distance_to_edge( label(_cellmask) )


# _lyso = organelles[0].sum(axis=0) 

_lyso = (apply_mask( organelles[0],cellmask_obj)>0).sum(axis=0)




In [19]:
# colors = color_labels(label(_cellmask) + label(_nuclei))
# d_to_edge = distance_to_edge( colors )


In [12]:

# viewer =napari.view_image(d_to_edge)
# viewer.add_image(_nuclei)
# viewer.add_image(colors)
viewer.add_image(_cellmask>0)
viewer.add_image(_lyso)



<Image layer '_lyso [1]' at 0x1ba376d90>

In [22]:
Zflat_cellmask = cellmask_obj.sum(axis=(1,2)) 
Zflat_nuclei = apply_mask(nuclei_obj,cellmask_obj).sum(axis=(1,2)) 

Zflat_nuclei

array([     0, 106014, 145230, 174336, 187272, 195216, 196374, 193998,
       185172, 162624, 118242, 102036,  81024,  48546,   4422],
      dtype=uint64)

In [74]:
# csv_path = out_data_path / f"{o}_{meta_dict["file_name"].split('/')[-1].split('.')[0]}_stats.csv"
Path(meta_dict['file_name']).name

'ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed.czi'

In [29]:
from scipy.ndimage import maximum_position, center_of_mass
from scipy.ndimage import sum as ndi_sum

from scipy.sparse import coo_matrix

from infer_subc_2d.core.img import distance_to_edge, distance_transform_edt

import centrosome

C_SELF = "These objects"
C_CENTERS_OF_OTHER_V2 = "Other objects"
C_CENTERS_OF_OTHER = "Centers of other objects"
C_EDGES_OF_OTHER = "Edges of other objects"
C_ALL = [C_SELF, C_CENTERS_OF_OTHER, C_EDGES_OF_OTHER]
Z_NONE = "None"
Z_MAGNITUDES = "Magnitudes only"
Z_MAGNITUDES_AND_PHASE = "Magnitudes and phase"
Z_ALL = [Z_NONE, Z_MAGNITUDES, Z_MAGNITUDES_AND_PHASE]

M_CATEGORY = "RadialDistribution"
F_FRAC_AT_D = "FracAtD"
F_MEAN_FRAC = "MeanFrac"
F_RADIAL_CV = "RadialCV"
F_ALL = [F_FRAC_AT_D, F_MEAN_FRAC, F_RADIAL_CV]

FF_SCALE = "%dof%d"
FF_OVERFLOW = "Overflow"
FF_GENERIC = "_%s_" + FF_SCALE
FF_FRAC_AT_D = F_FRAC_AT_D + FF_GENERIC
FF_MEAN_FRAC = F_MEAN_FRAC + FF_GENERIC
FF_RADIAL_CV = F_RADIAL_CV + FF_GENERIC

FF_ZERNIKE_MAGNITUDE = "ZernikeMagnitude"
FF_ZERNIKE_PHASE = "ZernikePhase"

MF_FRAC_AT_D = "_".join((M_CATEGORY, FF_FRAC_AT_D))
MF_MEAN_FRAC = "_".join((M_CATEGORY, FF_MEAN_FRAC))
MF_RADIAL_CV = "_".join((M_CATEGORY, FF_RADIAL_CV))
OF_FRAC_AT_D = "_".join((M_CATEGORY, F_FRAC_AT_D, "%s", FF_OVERFLOW))
OF_MEAN_FRAC = "_".join((M_CATEGORY, F_MEAN_FRAC, "%s", FF_OVERFLOW))
OF_RADIAL_CV = "_".join((M_CATEGORY, F_RADIAL_CV, "%s", FF_OVERFLOW))


In [30]:

def centers_of_labels(labels):
    """Return the i,j coordinates of the centers of a labels matrix
    
    The result returned is an 2 x n numpy array where n is the number
    of the label minus one, result[0,x] is the i coordinate of the center
    and result[x,1] is the j coordinate of the center.
    You can unpack the result as "i,j = centers_of_labels(labels)"
    """
    max_labels = np.max(labels)
    if max_labels == 0:
        return np.zeros((2, 0), int)

    result = center_of_mass(
        np.ones(labels.shape), labels, np.arange(max_labels) + 1
    )
    result = np.array(result)
    if result.ndim == 1:
        result.shape = (2, 1)
        return result
    return result.transpose()


In [31]:


def maximum_position_of_labels(image, labels, indices):
    """Return the i,j coordinates of the maximum value within each object
    
    image - measure the maximum within this image
    labels - use the objects within this labels matrix
    indices - label #s to measure
    
    The result returned is an 2 x n numpy array where n is the number
    of the label minus one, result[0,x] is the i coordinate of the center
    and result[x,1] is the j coordinate of the center.
    """

    if len(indices) == 0:
        return np.zeros((2, 0), int)

    result = maximum_position(image, labels, indices)
    result = np.array(result, int)
    if result.ndim == 1:
        result.shape = (2, 1)
        return result
    return result.transpose()


In [32]:


def size_similarly(labels, secondary):
    """Size the secondary matrix similarly to the labels matrix

    labels - labels matrix
    secondary - a secondary image or labels matrix which might be of
                different size.
    Return the resized secondary matrix and a mask indicating what portion
    of the secondary matrix is bogus (manufactured values).

    Either the mask is all ones or the result is a copy, so you can
    modify the output within the unmasked region w/o destroying the original.
    """
    if labels.shape[:2] == secondary.shape[:2]:
        return secondary, np.ones(secondary.shape, bool)
    if labels.shape[0] <= secondary.shape[0] and labels.shape[1] <= secondary.shape[1]:
        if secondary.ndim == 2:
            return (
                secondary[: labels.shape[0], : labels.shape[1]],
                np.ones(labels.shape, bool),
            )
        else:
            return (
                secondary[: labels.shape[0], : labels.shape[1], :],
                np.ones(labels.shape, bool),
            )

    # Some portion of the secondary matrix does not cover the labels
    result = np.zeros(
        list(labels.shape) + list(secondary.shape[2:]), secondary.dtype
    )
    i_max = min(secondary.shape[0], labels.shape[0])
    j_max = min(secondary.shape[1], labels.shape[1])
    if secondary.ndim == 2:
        result[:i_max, :j_max] = secondary[:i_max, :j_max]
    else:
        result[:i_max, :j_max, :] = secondary[:i_max, :j_max, :]
    mask = np.zeros(labels.shape, bool)
    mask[:i_max, :j_max] = 1
    return result, mask



In [33]:

def fixup_scipy_ndimage_result(whatever_it_returned):
    """Convert a result from scipy.ndimage to a numpy array
    
    scipy.ndimage has the annoying habit of returning a single, bare
    value instead of an array if the indexes passed in are of length 1.
    For instance:
    scind.maximum(image, labels, [1]) returns a float
    but
    scind.maximum(image, labels, [1,2]) returns a list
    """
    if getattr(whatever_it_returned, "__getitem__", False):
        return np.array(whatever_it_returned)
    else:
        return np.array([whatever_it_returned])


In [34]:
from centrosome import propagate


# get the flattened
_cellmask = cellmask_obj.sum(axis=0) 
_nuclei = apply_mask(nuclei_obj,cellmask_obj).sum(axis=0)
d_to_edge = distance_to_edge( label(_cellmask) )


# _lyso = organelles[0].sum(axis=0) 

_lyso = (apply_mask( organelles[0],cellmask_obj)>0).sum(axis=0)


organelle_ = organelles[0]
organelle_name =organelle_names[0]
intensity_img = apply_mask(img_data[0], cyto_mask)


n_bins = 6
scale_bins = True 
center_choice = C_EDGES_OF_OTHER


max_radius = 60 # not used

center_object_name = nuclei_obj if nuclei_obj is None else "nuclei" 


# get the flattened
_cellmask = (cellmask_obj>0).sum(axis=0) 
_nuclei = (apply_mask(nuclei_obj,cellmask_obj)>0).sum(axis=0)

_org = (organelle_>0).sum(axis=0) 
_img = intensity_img.sum(axis=0) 

nobjects = label(organelle_>0).max()


In [45]:

labels = (_cellmask>0).astype(np.uint16)

d_to_edge = distance_to_edge(labels) # made a local version

center_objects = (_nuclei>0).astype(np.uint16)


center_labels, cmask = size_similarly(labels, center_objects)
pixel_counts = fixup_scipy_ndimage_result(
    ndi_sum(
        np.ones(center_labels.shape),
        center_labels,
        np.arange(
            1, np.max(center_labels) + 1, dtype=np.int32
        ),
    )
)

good = pixel_counts > 0
i, j = ( centers_of_labels(center_labels) + 0.5).astype(int)
ig = i[good]
jg = j[good]
lg = np.arange(1, len(i) + 1)[good]
if (center_choice == C_CENTERS_OF_OTHER):  # Reduce the propagation labels to the centers of the centering objects
    center_labels = np.zeros(center_labels.shape, int)
    center_labels[ig, jg] = lg

cl, d_from_center = propagate.propagate(  np.zeros(center_labels.shape), center_labels, labels != 0, 1)
cl[labels == 0] = 0            # Erase the centers that fall outside of labels


# If objects are hollow or crescent-shaped, there may be objects without center labels. As a backup, find the
# center that is the closest to the center of mass.
missing_mask = (labels != 0) & (cl == 0)
missing_labels = np.unique(labels[missing_mask])

if len(missing_labels):
    all_centers = centers_of_labels(labels)
    missing_i_centers, missing_j_centers = all_centers[:, missing_labels-1]
    di = missing_i_centers[:, np.newaxis] - ig[np.newaxis, :]
    dj = missing_j_centers[:, np.newaxis] - jg[np.newaxis, :]
    missing_best = lg[np.argsort(di * di + dj * dj)[:, 0]]
    best = np.zeros(np.max(labels) + 1, int)
    best[missing_labels] = missing_best
    cl[missing_mask] = best[labels[missing_mask]]

    # Now compute the crow-flies distance to the centers of these pixels from whatever center was assigned to the object.
    iii, jjj = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
    di = iii[missing_mask] - i[cl[missing_mask] - 1]
    dj = jjj[missing_mask] - j[cl[missing_mask] - 1]
    d_from_center[missing_mask] = np.sqrt(di * di + dj * dj)

# # ELSE     if center_objects is  None:
# i, j = maximum_position_of_labels(   d_to_edge, labels, objects.indices )
# center_labels = np.zeros(labels.shape, int)
# center_labels[i, j] = labels[i, j]
# # Use the coloring trick here to process touching objectsin separate operations
# colors = color_labels(labels)
# ncolors = np.max(colors)
# d_from_center = np.zeros(labels.shape)
# cl = np.zeros(labels.shape, int)

# for color in range(1, ncolors + 1):
#     mask = colors == color
#     l, d = centrosome.propagate.propagate( np.zeros(center_labels.shape), center_labels, mask, 1)
#     d_from_center[mask] = d[mask]
#     cl[mask] = l[mask]




In [ ]:
good_mask = cl > 0

if (center_choice == C_EDGES_OF_OTHER):
    # Exclude pixels within the centering objects
    # when performing calculations from the centers
    good_mask = good_mask & (center_labels == 0)
i_center = np.zeros(cl.shape)
i_center[good_mask] = i[cl[good_mask] - 1]
j_center = np.zeros(cl.shape)
j_center[good_mask] = j[cl[good_mask] - 1]
normalized_distance = np.zeros(labels.shape)

if wants_scaled:
    total_distance = d_from_center + d_to_edge
    normalized_distance[good_mask] = d_from_center[good_mask] / ( total_distance[good_mask] + 0.001 )
else:
    normalized_distance[good_mask] = (d_from_center[good_mask] / maximum_radius)

dd[name] = [normalized_distance, i_center, j_center, good_mask]




#########  if / else
ngood_pixels = np.sum(good_mask)
good_labels = labels[good_mask]
bin_indexes = (normalized_distance * bin_count).astype(int)
bin_indexes[bin_indexes > bin_count] = bin_count
labels_and_bins = (good_labels - 1, bin_indexes[good_mask])

histogram = coo_matrix( (pixel_data[good_mask], labels_and_bins), (nobjects, bin_count + 1) ).toarray()

sum_by_object = np.sum(histogram, 1)
sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
fraction_at_distance = histogram / sum_by_object_per_bin
number_at_distance = coo_matrix)(np.ones(ngood_pixels), labels_and_bins), (nobjects, bin_count + 1)).toarray()

object_mask = number_at_distance > 0
sum_by_object = np.sum(number_at_distance, 1)
sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
fraction_at_bin = number_at_distance / sum_by_object_per_bin
mean_pixel_fraction = fraction_at_distance / ( fraction_at_bin + np.finfo(float).eps    )
masked_fraction_at_distance = np.ma.masked_array( fraction_at_distance, ~object_mask )
masked_mean_pixel_fraction = np.ma.masked_array(mean_pixel_fraction, ~object_mask)

# Anisotropy calculation.  Split each cell into eight wedges, then compute coefficient of variation of the wedges' mean intensities
# in each ring. Compute each pixel's delta from the center object's centroid
i, j = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
imask = i[good_mask] > i_center[good_mask]
jmask = j[good_mask] > j_center[good_mask]
absmask = abs(i[good_mask] - i_center[good_mask]) > abs(
    j[good_mask] - j_center[good_mask]
)
radial_index = (
    imask.astype(int) + jmask.astype(int) * 2 + absmask.astype(int) * 4
)
statistics = []

for bin in range(bin_count + (0 if wants_scaled else 1)):
    bin_mask = good_mask & (bin_indexes == bin)
    bin_pixels = np.sum(bin_mask)
    bin_labels = labels[bin_mask]
    bin_radial_index = radial_index[bin_indexes[good_mask] == bin]
    labels_and_radii = (bin_labels - 1, bin_radial_index)
    radial_values = coo_matrix( (pixel_data[bin_mask], labels_and_radii), (nobjects, 8) ).toarray()
    pixel_count = coo_matrix( (np.ones(bin_pixels), labels_and_radii), (nobjects, 8) ).toarray()

    mask = pixel_count == 0
    radial_means = np.ma.masked_array(radial_values / pixel_count, mask)
    radial_cv = np.std(radial_means, 1) / np.mean(radial_means, 1)
    radial_cv[np.sum(~mask, 1) == 0] = 0

    for measurement, feature, overflow_feature in (
        (fraction_at_distance[:, bin], MF_FRAC_AT_D, OF_FRAC_AT_D),
        (mean_pixel_fraction[:, bin], MF_MEAN_FRAC, OF_MEAN_FRAC),
        (np.array(radial_cv), MF_RADIAL_CV, OF_RADIAL_CV),
    ):
        if bin == bin_count:
            measurement_name = overflow_feature % image_name
        else:
            measurement_name = feature % (image_name, bin + 1, bin_count)

        measurements.add_measurement(object_name, measurement_name, measurement)

        # if feature in heatmaps:
        #     heatmaps[feature][bin_mask] = measurement[bin_labels - 1]

    radial_cv.mask = np.sum(~mask, 1) == 0
    bin_name = str(bin + 1) if bin < bin_count else "Overflow"

    statistics += [
        (image_name,
            object_name,
            bin_name,
            str(bin_count),
            np.round(np.mean(masked_fraction_at_distance[:, bin]), 4),
            np.round(np.mean(masked_mean_pixel_fraction[:, bin]), 4),
            np.round(np.mean(radial_cv), 4) )
    ]


In [46]:
viewer.add_image(cl)

<Image layer 'cl [1]' at 0x1b8ae4af0>

In [ ]:
from typing import Union

def get_radial_distribution(
        cellmask_obj: np.ndarray,
        nuclei_obj: Union[np.ndarray, None],
        organelle_:np.ndarray,
        organelle_name: str,
        intensity_img: np.ndarray,
    ):
    """Perform the radial measurements on the image set

    Parameters
    ------------
    cellmask_obj: np.ndarray,
    nuclei_obj: Union[np.ndarray, None],
    organelle_:np.ndarray,
    organelle_name: str,
    intensity_img: np.ndarray,
    
    Returns
    -------------
    returns one statistics tuple per ring.
    """
    # TODO: make these arguments
    n_bins = 6
    scale_bins = True 
    
    max_radius = 60 # not used
    
    center_object_name = nuclei_obj if nuclie_obj is None else "nuclei" 

    # get the flattened
    _cellmask = (cellmask_obj>0).sum(axis=0) 
    _nuclei = (apply_mask(nuclei_obj,cellmask_obj)>0).sum(axis=0)
    _org = (organelle_>0).sum(axis=0) 

    nobjects = np.max(organelle_)

    labels = _cellmask

    d_to_edge = distance_to_edge(labels) # made a local version
    center_objects = _nuclei

    if center_objects is not None:
        #
        # Use the center of the centering objects to assign a center
        # to each labeled pixel using propagation
        #

        center_labels, cmask = size_similarly(labels, center_objects.segmented)
        pixel_counts = fixup_scipy_ndimage_result(
            ndi_sum(
                np.ones(center_labels.shape),
                center_labels,
                np.arange(
                    1, np.max(center_labels) + 1, dtype=np.int32
                ),
            )
        )

        good = pixel_counts > 0
        i, j = ( centers_of_labels(center_labels) + 0.5).astype(int)
        ig = i[good]
        jg = j[good]
        lg = np.arange(1, len(i) + 1)[good]
        if (center_choice == C_CENTERS_OF_OTHERS):  # Reduce the propagation labels to the centers of the centering objects
            center_labels = np.zeros(center_labels.shape, int)
            center_labels[ig, jg] = lg

        cl, d_from_center = centrosome.propagate.propagate(  np.zeros(center_labels.shape), center_labels, labels != 0, 1)
        cl[labels == 0] = 0            # Erase the centers that fall outside of labels

        # If objects are hollow or crescent-shaped, there may be objects without center labels. As a backup, find the
        # center that is the closest to the center of mass.
        missing_mask = (labels != 0) & (cl == 0)
        missing_labels = np.unique(labels[missing_mask])

        if len(missing_labels):
            all_centers = centers_of_labels(labels)
            missing_i_centers, missing_j_centers = all_centers[:, missing_labels-1]
            di = missing_i_centers[:, np.newaxis] - ig[np.newaxis, :]
            dj = missing_j_centers[:, np.newaxis] - jg[np.newaxis, :]
            missing_best = lg[np.argsort(di * di + dj * dj)[:, 0]]
            best = np.zeros(np.max(labels) + 1, int)
            best[missing_labels] = missing_best
            cl[missing_mask] = best[labels[missing_mask]]

            # Now compute the crow-flies distance to the centers of these pixels from whatever center was assigned to the object.
            iii, jjj = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
            di = iii[missing_mask] - i[cl[missing_mask] - 1]
            dj = jjj[missing_mask] - j[cl[missing_mask] - 1]
            d_from_center[missing_mask] = np.sqrt(di * di + dj * dj)
    else:
        # Find the point in each object farthest away from the edge.
        # This does better than the centroid:
        # * The center is within the object
        # * The center tends to be an interesting point, like the center of the nucleus or the center of one or the other of two touching cells.
        i, j = maximum_position_of_labels(   d_to_edge, labels, objects.indices )
        center_labels = np.zeros(labels.shape, int)
        center_labels[i, j] = labels[i, j]
        # Use the coloring trick here to process touching objectsin separate operations
        colors = color_labels(labels)
        ncolors = np.max(colors)
        d_from_center = np.zeros(labels.shape)
        cl = np.zeros(labels.shape, int)

        for color in range(1, ncolors + 1):
            mask = colors == color
            l, d = centrosome.propagate.propagate( np.zeros(center_labels.shape), center_labels, mask, 1)
            d_from_center[mask] = d[mask]
            cl[mask] = l[mask]

        good_mask = cl > 0

        if (center_choice == C_EDGES_OF_OTHER):
            # Exclude pixels within the centering objects
            # when performing calculations from the centers
            good_mask = good_mask & (center_labels == 0)
        i_center = np.zeros(cl.shape)
        i_center[good_mask] = i[cl[good_mask] - 1]
        j_center = np.zeros(cl.shape)
        j_center[good_mask] = j[cl[good_mask] - 1]
        normalized_distance = np.zeros(labels.shape)

        if wants_scaled:
            total_distance = d_from_center + d_to_edge
            normalized_distance[good_mask] = d_from_center[good_mask] / ( total_distance[good_mask] + 0.001 )
        else:
            normalized_distance[good_mask] = (d_from_center[good_mask] / maximum_radius)
        dd[name] = [normalized_distance, i_center, j_center, good_mask]



    ngood_pixels = np.sum(good_mask)
    good_labels = labels[good_mask]
    bin_indexes = (normalized_distance * bin_count).astype(int)
    bin_indexes[bin_indexes > bin_count] = bin_count
    labels_and_bins = (good_labels - 1, bin_indexes[good_mask])

    histogram = coo_matrix( (pixel_data[good_mask], labels_and_bins), (nobjects, bin_count + 1) ).toarray()

    sum_by_object = np.sum(histogram, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_distance = histogram / sum_by_object_per_bin
    number_at_distance = coo_matrix)(np.ones(ngood_pixels), labels_and_bins), (nobjects, bin_count + 1)).toarray()

    object_mask = number_at_distance > 0
    sum_by_object = np.sum(number_at_distance, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_bin = number_at_distance / sum_by_object_per_bin
    mean_pixel_fraction = fraction_at_distance / ( fraction_at_bin + np.finfo(float).eps    )
    masked_fraction_at_distance = np.ma.masked_array( fraction_at_distance, ~object_mask )
    masked_mean_pixel_fraction = np.ma.masked_array(mean_pixel_fraction, ~object_mask)

    # Anisotropy calculation.  Split each cell into eight wedges, then compute coefficient of variation of the wedges' mean intensities
    # in each ring. Compute each pixel's delta from the center object's centroid
    i, j = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
    imask = i[good_mask] > i_center[good_mask]
    jmask = j[good_mask] > j_center[good_mask]
    absmask = abs(i[good_mask] - i_center[good_mask]) > abs(
        j[good_mask] - j_center[good_mask]
    )
    radial_index = (
        imask.astype(int) + jmask.astype(int) * 2 + absmask.astype(int) * 4
    )
    statistics = []

    for bin in range(bin_count + (0 if wants_scaled else 1)):
        bin_mask = good_mask & (bin_indexes == bin)
        bin_pixels = np.sum(bin_mask)
        bin_labels = labels[bin_mask]
        bin_radial_index = radial_index[bin_indexes[good_mask] == bin]
        labels_and_radii = (bin_labels - 1, bin_radial_index)
        radial_values = coo_matrix( (pixel_data[bin_mask], labels_and_radii), (nobjects, 8) ).toarray()
        pixel_count = coo_matrix( (np.ones(bin_pixels), labels_and_radii), (nobjects, 8) ).toarray()

        mask = pixel_count == 0
        radial_means = np.ma.masked_array(radial_values / pixel_count, mask)
        radial_cv = np.std(radial_means, 1) / np.mean(radial_means, 1)
        radial_cv[np.sum(~mask, 1) == 0] = 0

        for measurement, feature, overflow_feature in (
            (fraction_at_distance[:, bin], MF_FRAC_AT_D, OF_FRAC_AT_D),
            (mean_pixel_fraction[:, bin], MF_MEAN_FRAC, OF_MEAN_FRAC),
            (np.array(radial_cv), MF_RADIAL_CV, OF_RADIAL_CV),
        ):
            if bin == bin_count:
                measurement_name = overflow_feature % image_name
            else:
                measurement_name = feature % (image_name, bin + 1, bin_count)
 
           measurements.add_measurement(object_name, measurement_name, measurement)

            # if feature in heatmaps:
            #     heatmaps[feature][bin_mask] = measurement[bin_labels - 1]

        radial_cv.mask = np.sum(~mask, 1) == 0
        bin_name = str(bin + 1) if bin < bin_count else "Overflow"

        statistics += [
            (image_name,
                object_name,
                bin_name,
                str(bin_count),
                np.round(np.mean(masked_fraction_at_distance[:, bin]), 4),
                np.round(np.mean(masked_mean_pixel_fraction[:, bin]), 4),
                np.round(np.mean(radial_cv), 4) )
        ]

    return statistics

In [18]:
from typing import Union

def get_radial_distribution(
        cellmask: np.ndarray,
        nuclei: Union[np.ndarray, None],
        organelle:np.ndarray,
        organelle_name: str,
    ):
    """Perform the radial measurements on the image set

    cellmask
    image_name - make measurements on this image
    object_name - make measurements on these objects
    center_object_name - if None use the centers of these related objects as
                    the centers for radial measurements. None to use the
                    objects themselves.
    nuclei -  if none use the user's center choice for this object:
                    C_SELF, C_CENTERS_OF_OBJECTS or C_EDGES_OF_OBJECTS.
    bin_count_settings - the bin count settings group
    dd - a dictionary for saving reusable partial results

    returns one statistics tuple per ring.
    """
    # TODO: make these arguments
    n_bins = 5
    
    bin_count = bin_count_settings.bin_count.value
    wants_scaled = bin_count_settings.wants_scaled.value


    maximum_radius = bin_count_settings.maximum_radius.value
    image = workspace.image_set.get_image(image_name, must_be_grayscale=True)
    objects = workspace.object_set.get_objects(object_name)
    labels, pixel_data = crop_labels_and_image(objects.segmented, image.pixel_data)

    nobjects = np.max(organelle)


    d_to_edge = distance_to_edge(labels) # made a local version
    if center_object_name is not None:
        #
        # Use the center of the centering objects to assign a center
        # to each labeled pixel using propagation
        #
        center_objects = workspace.object_set.get_objects(center_object_name)

        center_labels, cmask = size_similarly(labels, center_objects.segmented)
        pixel_counts = fixup_scipy_ndimage_result(
            ndi_sum(
                np.ones(center_labels.shape),
                center_labels,
                np.arange(
                    1, np.max(center_labels) + 1, dtype=np.int32
                ),
            )
        )

        good = pixel_counts > 0
        i, j = ( centers_of_labels(center_labels) + 0.5).astype(int)
        ig = i[good]
        jg = j[good]
        lg = np.arange(1, len(i) + 1)[good]
        if (center_choice == C_CENTERS_OF_OTHERS):  # Reduce the propagation labels to the centers of the centering objects
            center_labels = np.zeros(center_labels.shape, int)
            center_labels[ig, jg] = lg

        cl, d_from_center = centrosome.propagate.propagate(  np.zeros(center_labels.shape), center_labels, labels != 0, 1)
        cl[labels == 0] = 0            # Erase the centers that fall outside of labels

        # If objects are hollow or crescent-shaped, there may be objects without center labels. As a backup, find the
        # center that is the closest to the center of mass.
        missing_mask = (labels != 0) & (cl == 0)
        missing_labels = np.unique(labels[missing_mask])

        if len(missing_labels):
            all_centers = centers_of_labels(labels)
            missing_i_centers, missing_j_centers = all_centers[:, missing_labels-1]
            di = missing_i_centers[:, np.newaxis] - ig[np.newaxis, :]
            dj = missing_j_centers[:, np.newaxis] - jg[np.newaxis, :]
            missing_best = lg[np.argsort(di * di + dj * dj)[:, 0]]
            best = np.zeros(np.max(labels) + 1, int)
            best[missing_labels] = missing_best
            cl[missing_mask] = best[labels[missing_mask]]

            # Now compute the crow-flies distance to the centers of these pixels from whatever center was assigned to the object.
            iii, jjj = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
            di = iii[missing_mask] - i[cl[missing_mask] - 1]
            dj = jjj[missing_mask] - j[cl[missing_mask] - 1]
            d_from_center[missing_mask] = np.sqrt(di * di + dj * dj)
    else:
        # Find the point in each object farthest away from the edge.
        # This does better than the centroid:
        # * The center is within the object
        # * The center tends to be an interesting point, like the center of the nucleus or the center of one or the other of two touching cells.
        i, j = maximum_position_of_labels(   d_to_edge, labels, objects.indices )
        center_labels = np.zeros(labels.shape, int)
        center_labels[i, j] = labels[i, j]
        # Use the coloring trick here to process touching objectsin separate operations
        colors = color_labels(labels)
        ncolors = np.max(colors)
        d_from_center = np.zeros(labels.shape)
        cl = np.zeros(labels.shape, int)

        for color in range(1, ncolors + 1):
            mask = colors == color
            l, d = centrosome.propagate.propagate( np.zeros(center_labels.shape), center_labels, mask, 1)
            d_from_center[mask] = d[mask]
            cl[mask] = l[mask]

        good_mask = cl > 0

        if (center_choice == C_EDGES_OF_OTHER):
            # Exclude pixels within the centering objects
            # when performing calculations from the centers
            good_mask = good_mask & (center_labels == 0)
        i_center = np.zeros(cl.shape)
        i_center[good_mask] = i[cl[good_mask] - 1]
        j_center = np.zeros(cl.shape)
        j_center[good_mask] = j[cl[good_mask] - 1]
        normalized_distance = np.zeros(labels.shape)

        if wants_scaled:
            total_distance = d_from_center + d_to_edge
            normalized_distance[good_mask] = d_from_center[good_mask] / ( total_distance[good_mask] + 0.001 )
        else:
            normalized_distance[good_mask] = (d_from_center[good_mask] / maximum_radius)
        dd[name] = [normalized_distance, i_center, j_center, good_mask]



    ngood_pixels = np.sum(good_mask)
    good_labels = labels[good_mask]
    bin_indexes = (normalized_distance * bin_count).astype(int)
    bin_indexes[bin_indexes > bin_count] = bin_count
    labels_and_bins = (good_labels - 1, bin_indexes[good_mask])

    histogram = coo_matrix( (pixel_data[good_mask], labels_and_bins), (nobjects, bin_count + 1) ).toarray()

    sum_by_object = np.sum(histogram, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_distance = histogram / sum_by_object_per_bin
    number_at_distance = coo_matrix)(np.ones(ngood_pixels), labels_and_bins), (nobjects, bin_count + 1)).toarray()

    object_mask = number_at_distance > 0
    sum_by_object = np.sum(number_at_distance, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_bin = number_at_distance / sum_by_object_per_bin
    mean_pixel_fraction = fraction_at_distance / ( fraction_at_bin + np.finfo(float).eps    )
    masked_fraction_at_distance = np.ma.masked_array( fraction_at_distance, ~object_mask )
    masked_mean_pixel_fraction = np.ma.masked_array(mean_pixel_fraction, ~object_mask)

    # Anisotropy calculation.  Split each cell into eight wedges, then compute coefficient of variation of the wedges' mean intensities
    # in each ring. Compute each pixel's delta from the center object's centroid
    i, j = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
    imask = i[good_mask] > i_center[good_mask]
    jmask = j[good_mask] > j_center[good_mask]
    absmask = abs(i[good_mask] - i_center[good_mask]) > abs(
        j[good_mask] - j_center[good_mask]
    )
    radial_index = (
        imask.astype(int) + jmask.astype(int) * 2 + absmask.astype(int) * 4
    )
    statistics = []

    for bin in range(bin_count + (0 if wants_scaled else 1)):
        bin_mask = good_mask & (bin_indexes == bin)
        bin_pixels = np.sum(bin_mask)
        bin_labels = labels[bin_mask]
        bin_radial_index = radial_index[bin_indexes[good_mask] == bin]
        labels_and_radii = (bin_labels - 1, bin_radial_index)
        radial_values = coo_matrix( (pixel_data[bin_mask], labels_and_radii), (nobjects, 8) ).toarray()
        pixel_count = coo_matrix( (np.ones(bin_pixels), labels_and_radii), (nobjects, 8) ).toarray()

        mask = pixel_count == 0
        radial_means = np.ma.masked_array(radial_values / pixel_count, mask)
        radial_cv = np.std(radial_means, 1) / np.mean(radial_means, 1)
        radial_cv[np.sum(~mask, 1) == 0] = 0

        for measurement, feature, overflow_feature in (
            (fraction_at_distance[:, bin], MF_FRAC_AT_D, OF_FRAC_AT_D),
            (mean_pixel_fraction[:, bin], MF_MEAN_FRAC, OF_MEAN_FRAC),
            (np.array(radial_cv), MF_RADIAL_CV, OF_RADIAL_CV),
        ):
            if bin == bin_count:
                measurement_name = overflow_feature % image_name
            else:
                measurement_name = feature % (image_name, bin + 1, bin_count)
 
           measurements.add_measurement(object_name, measurement_name, measurement)

            # if feature in heatmaps:
            #     heatmaps[feature][bin_mask] = measurement[bin_labels - 1]

        radial_cv.mask = np.sum(~mask, 1) == 0
        bin_name = str(bin + 1) if bin < bin_count else "Overflow"

        statistics += [
            (image_name,
                object_name,
                bin_name,
                str(bin_count),
                np.round(np.mean(masked_fraction_at_distance[:, bin]), 4),
                np.round(np.mean(masked_mean_pixel_fraction[:, bin]), 4),
                np.round(np.mean(radial_cv), 4) )
        ]

    return statistics

SyntaxError: unmatched ')' (2857032193.py, line 143)

In [ ]:
###################
# MITOCONDRIA
###################
mito_obj  = get_mito(img_data,meta_dict, out_data_path)
mito_table = get_summary_stats_3D( mito_obj, img_data[MITO_CH],cytoplasm_mask)

###################
#  GOLGI
###################
golgi_obj = get_golgi(img_data,meta_dict, out_data_path)
golgi_obj = get_summary_stats_3D( golgi_obj, img_data[GOLGI_CH],cytoplasm_mask)

###################
#  PEROXISOME
###################
perox_obj  = get_perox(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er_obj  = get_ER(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
LD_obj  =  get_LD(img_data,meta_dict, out_data_path)

In [ ]:

viewer = napari.view_image(lyso_obj)

In [ ]:
viewer.add_image(florescence)
viewer.add_image(cytoplasm_mask)


In [ ]:
from math import pi as PI

table['equivalent_diameter'], table['area'], (2 * 3 *  table['area'] / PI) ** (1 /3)


In [ ]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  infer_and_export_nuclei(img_data,meta_dict, out_data_path)

soma_obj = infer_and_export_soma(img_data, nuclei,meta_dict, out_data_path)
cytoplasm_mask =  infer_and_export_cytoplasm(soma_obj, nuclei_obj, meta_dict, out_data_path)



In [ ]:
viewer.add_image(cytoplasm_mask)


In [ ]:
###################
# LYSOSOME
###################
lyso_obj  = get_lyso(img_data,meta_dict, out_data_path)
florescence = apply_mask(img_data[LYSO_CH],cytoplasm_mask )  
lyso_table = get_summary_stats_3D(lyso_obj, florescence)


In [ ]:

###################
# LYSOSOME
###################
lyso = infer_and_export_lyso(img_data,meta_dict, out_data_path)

###################
# MITOCONDRIA
###################
mitochondria = infer_and_export_mito(img_data,meta_dict, out_data_path)
###################
#  GOLGI
###################
golgi = infer_and_export_golgi(img_data,meta_dict, out_data_path)

###################
#  PEROXISOME
###################
peroxisome = infer_and_export_perox(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er = infer_and_export_ER(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid =  infer_and_export_LD(img_data,meta_dict, out_data_path)

In [ ]:
lipid

In [ ]:

# CHOOSE which inferred organelle to 
if organelle == NUC_CH: #0
    organelle_name = 'nucleus'
elif organelle == LYSO_CH: #1 
    organelle_name = 'lyso'
elif organelle == MITO_CH: #2
    organelle_name = 'mitochondria'
elif organelle == GOLGI_CH: #3
    organelle_name = 'golgi'
elif organelle == PEROX_CH: #4
    organelle_name = 'peroxisome'
elif organelle == ER_CH: #5
    organelle_name = 'ER'
elif organelle == LD_CH: #6 
    organelle_name = 'lipid'
elif organelle == RESIDUAL_CH: #7
    organelle_name = 'residual'

print(organelle_name)

target = segmentation[organelle].squeeze()
target = apply_mask(target, mask.squeeze())    
target_intensity = florescence[organelle].squeeze()

labels = label(target).astype("int").squeeze()


summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )
# target.shape, target_intensity.shape, labels.shape

labels.max()



----------------------

run a batch of ALL the images

First get all the images

now build a function to loop over them all and export




In [ ]:
output_tiffs = batch_process_all_czi(data_root_path)
